In [1]:
import requests
import urllib.request as ur
from bs4 import BeautifulSoup
import tqdm
import time

In [5]:
def text_preprocess(names:list) -> list:
    # mb smth more complex needed
    return [a.replace("@", "").lower() for a in names]

In [6]:
samp_url = 'https://topdeck.ru/forums/topic/7409-%D0%B1%D1%8B%D0%BB%D0%B8-%D0%BD%D0%B0%D0%BA%D0%B0%D0%B7%D0%B0%D0%BD%D1%8B/page/36/'

def get_dict_for_one_page(page_url:str) -> dict:
    '''
    makes a query, then parses it to return dict of "username": [punishment links]
    '''

    html = ur.urlopen(page_url)
    if html.status != 200:
        raise ValueError
    bs = BeautifulSoup(html.read())

    resp = bs.findAll('div', {"class": "ipsType_normal ipsType_richText ipsPadding_bottom ipsContained"})
    share = bs.findAll('div', {"class": "ipsType_reset ipsResponsive_hidePhone"})
    if len(resp) != len(share):
        print('more complicated link parser needed')
        raise NotImplemented
    
    ans_dict = {}

    for i, txt in enumerate(resp):
        mentioned_users = [u.text for u in txt.findAll("strong")] + [
            u.text
            for u in txt.findAll("a", {"contenteditable": "false"})
        ]
        mentioned_users = text_preprocess(mentioned_users)
        link = share[i].a["href"]
        if len(mentioned_users) == 0:
            print(f'No users mentioned in message {link}')

        for user in mentioned_users:
            if user in ans_dict:
                ans_dict[user].append(link)
            else:
                ans_dict[user] = [link]

    #to make order chronological
    for k in ans_dict:
        ans_dict[k] = ans_dict[k][::-1]

    return ans_dict

get_dict_for_one_page(samp_url)

{'deutchland magic': ['https://topdeck.ru/forums/topic/7409-%D0%B1%D1%8B%D0%BB%D0%B8-%D0%BD%D0%B0%D0%BA%D0%B0%D0%B7%D0%B0%D0%BD%D1%8B/?do=findComment&comment=1629927'],
 'mijoil': ['https://topdeck.ru/forums/topic/7409-%D0%B1%D1%8B%D0%BB%D0%B8-%D0%BD%D0%B0%D0%BA%D0%B0%D0%B7%D0%B0%D0%BD%D1%8B/?do=findComment&comment=1629927'],
 'neeclejoose': ['https://topdeck.ru/forums/topic/7409-%D0%B1%D1%8B%D0%BB%D0%B8-%D0%BD%D0%B0%D0%BA%D0%B0%D0%B7%D0%B0%D0%BD%D1%8B/?do=findComment&comment=1629927'],
 'cardplace.ru': ['https://topdeck.ru/forums/topic/7409-%D0%B1%D1%8B%D0%BB%D0%B8-%D0%BD%D0%B0%D0%BA%D0%B0%D0%B7%D0%B0%D0%BD%D1%8B/?do=findComment&comment=1629927'],
 'илья шишкин': ['https://topdeck.ru/forums/topic/7409-%D0%B1%D1%8B%D0%BB%D0%B8-%D0%BD%D0%B0%D0%BA%D0%B0%D0%B7%D0%B0%D0%BD%D1%8B/?do=findComment&comment=1629927'],
 'никита солохин': ['https://topdeck.ru/forums/topic/7409-%D0%B1%D1%8B%D0%BB%D0%B8-%D0%BD%D0%B0%D0%BA%D0%B0%D0%B7%D0%B0%D0%BD%D1%8B/?do=findComment&comment=1629927'],
 'gorefist':

In [7]:
def dict_addition(d1:dict, d2:dict):
    '''
    dicts should be dicts of lists, mutates d1
    '''
    for key in d2:
        if key in d1:
            d1[key] += d2[key]
        else:
            d1[key] = d2[key]

d1 = {'ls':[0], 'sl':[1]} 
dict_addition(
    d1,
    {'ds':[0], 'ls':[1]}
)
d1

{'ls': [0, 1], 'sl': [1], 'ds': [0]}

In [8]:
# add reading of preparsed in future
start_dict = {}

In [9]:
def get_dict_with_retries(url, max_tries=9):
    tries = 0
    while tries < max_tries:
        try:
            return get_dict_for_one_page(url)
        except ValueError:
            print(f'parsing for url {url} failed, retrying')
            time.sleep(5)
            tries += 1
    raise ValueError


In [10]:
applicable_pages = range(37, 0, -1)

for page in tqdm.tqdm(applicable_pages):
    current_url = f'https://topdeck.ru/forums/topic/7409-%D0%B1%D1%8B%D0%BB%D0%B8-%D0%BD%D0%B0%D0%BA%D0%B0%D0%B7%D0%B0%D0%BD%D1%8B/page/{page}/'
    dict_for_page = get_dict_with_retries(current_url)
    dict_addition(start_dict, dict_for_page)

 41%|████      | 15/37 [00:38<01:13,  3.35s/it]

No users mentioned in message https://topdeck.ru/forums/topic/7409-%D0%B1%D1%8B%D0%BB%D0%B8-%D0%BD%D0%B0%D0%BA%D0%B0%D0%B7%D0%B0%D0%BD%D1%8B/?do=findComment&comment=1354644


 51%|█████▏    | 19/37 [00:47<00:43,  2.42s/it]

No users mentioned in message https://topdeck.ru/forums/topic/7409-%D0%B1%D1%8B%D0%BB%D0%B8-%D0%BD%D0%B0%D0%BA%D0%B0%D0%B7%D0%B0%D0%BD%D1%8B/?do=findComment&comment=1244807


 73%|███████▎  | 27/37 [01:04<00:22,  2.24s/it]

...

No users mentioned in message https://topdeck.ru/forums/topic/7409-%D0%B1%D1%8B%D0%BB%D0%B8-%D0%BD%D0%B0%D0%BA%D0%B0%D0%B7%D0%B0%D0%BD%D1%8B/?do=findComment&comment=290083


 97%|█████████▋| 36/37 [01:23<00:02,  2.06s/it]

No users mentioned in message https://topdeck.ru/forums/topic/7409-%D0%B1%D1%8B%D0%BB%D0%B8-%D0%BD%D0%B0%D0%BA%D0%B0%D0%B7%D0%B0%D0%BD%D1%8B/?do=findComment&comment=221431
No users mentioned in message https://topdeck.ru/forums/topic/7409-%D0%B1%D1%8B%D0%BB%D0%B8-%D0%BD%D0%B0%D0%BA%D0%B0%D0%B7%D0%B0%D0%BD%D1%8B/?do=findComment&comment=240801
No users mentioned in message https://topdeck.ru/forums/topic/7409-%D0%B1%D1%8B%D0%BB%D0%B8-%D0%BD%D0%B0%D0%BA%D0%B0%D0%B7%D0%B0%D0%BD%D1%8B/?do=findComment&comment=241043


100%|██████████| 37/37 [01:25<00:00,  2.31s/it]

No users mentioned in message https://topdeck.ru/forums/topic/7409-%D0%B1%D1%8B%D0%BB%D0%B8-%D0%BD%D0%B0%D0%BA%D0%B0%D0%B7%D0%B0%D0%BD%D1%8B/?do=findComment&comment=171975
No users mentioned in message https://topdeck.ru/forums/topic/7409-%D0%B1%D1%8B%D0%BB%D0%B8-%D0%BD%D0%B0%D0%BA%D0%B0%D0%B7%D0%B0%D0%BD%D1%8B/?do=findComment&comment=190325
No users mentioned in message https://topdeck.ru/forums/topic/7409-%D0%B1%D1%8B%D0%BB%D0%B8-%D0%BD%D0%B0%D0%BA%D0%B0%D0%B7%D0%B0%D0%BD%D1%8B/?do=findComment&comment=194262
No users mentioned in message https://topdeck.ru/forums/topic/7409-%D0%B1%D1%8B%D0%BB%D0%B8-%D0%BD%D0%B0%D0%BA%D0%B0%D0%B7%D0%B0%D0%BD%D1%8B/?do=findComment&comment=203929


In [11]:
users = list(zip(start_dict.keys(), [len(a) for a in start_dict.values()]))
users.sort(key=lambda x: -x[1])

In [12]:
users[:20]

[('goodspeed', 17),
 ('/dev/null', 12),
 ('mrm83', 9),
 ('salva', 7),
 ('padus', 6),
 ('exhume', 5),
 ('michael', 5),
 ('serg121', 5),
 ('sorcerec', 5),
 ('nortus', 4),
 ('badsanta', 4),
 ('vishen', 4),
 ('sorcerec ', 4),
 ('fritz', 4),
 ('eek1tweek', 3),
 ('pervitin_panzinger', 3),
 ('temiaj', 3),
 ('streatmagic', 3),
 ('farsmac', 3),
 ('gods of the war', 3)]

In [13]:
start_dict['thefamilyman']

['https://topdeck.ru/forums/topic/7409-%D0%B1%D1%8B%D0%BB%D0%B8-%D0%BD%D0%B0%D0%BA%D0%B0%D0%B7%D0%B0%D0%BD%D1%8B/?do=findComment&comment=1144154']

In [14]:
buffer = ''

for k in tqdm.tqdm(start_dict):
    buffer += f'{k}:\n'
    buffer += f'{start_dict[k]}\n'

out_file = open('banned_users.txt', 'w')
out_file.write(buffer)
out_file.close()

100%|██████████| 1046/1046 [00:00<00:00, 281775.34it/s]
